In [37]:
import scipy.io
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

In [38]:
def load_data(file):
    mat = scipy.io.loadmat(file)
    return mat['val'][0]

In [39]:
def show_single_plot(y):
    electrode0=y[0]
    electrode1=y[1]
    electrode2=y[2]
    electrode3=y[3]

    fig,ax = plt.subplots(2,2)
    x=(np.arange(0,len(y[0])))

    ax[0][0].plot(x,y[0],linewidth=.4)
    ax[0][1].plot(x,y[1],linewidth=.4)
    ax[1][0].plot(x,y[2],linewidth=.4)
    ax[1][1].plot(x,y[3],linewidth=.4)


In [55]:
def filter_fetus(y_beat):
#separating heartbeats
    first_beats = scipy.signal.find_peaks(y_beat[0], height=0.05)
    second_beats = scipy.signal.find_peaks(y_beat[1], height=0.05)
    fetus =  y_beat[0] if first_beats[0].shape[0] > second_beats[0].shape[0] else y_beat[1]
    fetal_info= first_beats if first_beats[0].shape[0] > second_beats[0].shape[0] else second_beats
    fetal_spike_time=fetal_info[0]
    fetal_amplitude=fetal_info[1]['peak_heights']
    #print(fetal_amplitude)
    #print(fetal_spike_time)

    return fetus,fetal_amplitude,fetal_spike_time


In [41]:
def find_anomalies(data):
    #define a list to accumlate anomalies
    anomalies = []
    
    # Set upper and lower limit to 3 standard deviation
    data_std = np.std(data)
    data_mean = np.mean(data)
    cuttoff=3 # 68% -1 , 95% -2 , 99.7% -3
    anomaly_cut_off = data_std * cuttoff
    
    lower_limit  = data_mean - anomaly_cut_off 
    upper_limit = data_mean + anomaly_cut_off
    
    # Generate outliers
    for outlier in data:
        if outlier > upper_limit or outlier < lower_limit:
            anomalies.append(outlier)
    
    return anomalies


In [50]:
def filter_data(y):
    transformer = FastICA(whiten='unit-variance')
    y_transposed = y.T
    y_transformed = transformer.fit_transform(y_transposed)
    beats = []
    for row in y_transformed.T:
        if min(row)< -0.05 or max(row)>0.05:
            # flip sign
            if abs(np.min(row)) > abs(np.max(row)):
                row = row * -1
            beats.append(row)
    beats = np.asarray(beats)

    print(beats.shape)
    return beats

In [57]:
def main():
    start = 0
    stop = 2 #set the number of files you want to go through
    for i, filename in enumerate(os.listdir('heartbeat_data')):
        out = re.findall(r'\d+', filename)
        if int(out[0]) < start:
            continue
        if i == stop:
            break
        
        y=load_data('heartbeat_data/'+filename)
        y_beats=filter_data(y)
        fetus,fetal_amplitude,fetal_spike_time=filter_fetus(y_beats)
        #sns.boxplot(data=fetal_amplitude)
        anomaly1=find_anomalies(fetal_amplitude)
        if anomaly1!=[]:
            print("irregular heartbeat anomaly 1",filename,anomaly1)
            print(filename,anomaly1)
        else:
            print(" no anomaly 1 irregularities heartbeat in ",filename)
        
        #calculaing widh between peaks
            
        n=fetal_spike_time.shape[0]
        peak_width_diff=[np.abs(fetal_spike_time[i]-fetal_spike_time[i+1]) for i in range(n-5)]
        #sns.boxplot(data=peak_width_diff)
        anomaly2=find_anomalies(peak_width_diff)
        if anomaly2!=[]:
            print("irregular heartbeat anomaly2",filename,anomaly2)
        else:
            print(" no anomaly 2 irregularities heartbeat in ",filename)
main()

(3, 1500)
 no anomaly 1 irregularities heartbeat in  000.mat
 no anomaly 2 irregularities heartbeat in  000.mat
(2, 2365)
 no anomaly 1 irregularities heartbeat in  001.mat
 no anomaly 2 irregularities heartbeat in  001.mat
